In [1]:
# Shelterluv Adoption & Length of Stay Analysis
# ---------------------------------------------
# Basic setup: load libraries and configure visuals

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Make plots look nice
sns.set(style="whitegrid", palette="muted", font_scale=1.1)

# Show all columns in pandas output
pd.set_option("display.max_columns", None)

In [2]:
# Load Data

## Load demographic data
df_demo = pd.read_csv("../data/SL_data_20251112.csv")

## Load length-of-stay/outcome data
df_los = pd.read_csv("../data/SL_length_of_stay_20251112.csv")

## Merge datasets on 'Animal ID'
df = pd.merge(df_demo, df_los, on='Animal ID', how='inner')

## Quick peak at data to get column names
df.head()

,Date Created,Intake Date_x,Animal ID,Name,Primary Breed_x,Secondary Breed_x,Sex,Age (Y/M/D),Age Group,Birthdate,Primary Color,Secondary Color,Pattern,Microchip Number,Microchip Issuer,Microchip Implant Date,Adoption Category,Current Location_x,Days In Custody,Current Status_x,Holdable,Adoptable,In Custody,Adoption Fee Group,Foster Person ID,Foster Person Name,Photo,Foster Notes,Animal Name,Species,Primary Breed_y,Secondary Breed_y,Date of Birth,Age Group at Intake,Current Status_y,Current Location_y,Intake Date_y,Intake Type,Outcome Date,Outcome Type,Days in Custody,Days Onsite,Days Available
0,06/20/2024,04/18/2024,CCL-A-1,Alice,Domestic Shorthair,—,Female,11Y/6M/26D,Senior Cat,4/16/2014,Torbie,NaN,NaN,NaN,—,—,—,Euthanased,88,Deceased,No,No,No,—,—,—,https://new-s3.shelterluv.com/profile-pictures...,—,Alice,Cat,Domestic Shorthair,NaN,4/16/2014,Senior Cat (8+ years),Deceased,Euthanased,4/18/2024,Stray In,7/14/2024,Euthanasia,88,88,88
1,06/26/2024,03/18/2024,CCL-A-2,Asher,Domestic Shorthair,—,Male,2Y/8M/0D,Adult Cat,3/12/2023,Black,NaN,NaN,992000003360734,NanoCHIP,—,Restrictions,Adopted: Sandra Decker,212,Healthy In Home,No,No,No,Bonded Pair,—,—,https://new-s3.shelterluv.com/profile-pictures...,—,Asher,Cat,Domestic Shorthair,NaN,3/12/2023,Adult Cat (5 months-8 years),Healthy In Home,Adopted: Sandra Decker,3/18/2024,Stray In,10/15/2024,Adoption,212,212,212
2,06/26/2024,06/03/2024,CCL-A-4,Criminal,Domestic Shorthair,—,Male,2Y/11M/27D,Adult Cat,11/15/2022,Orange,NaN,NaN,992000003226634,Other,—,—,Adopted: Shyanne Baker,108,Healthy In Home,No,No,No,Adult Cat,—,—,https://new-s3.shelterluv.com/profile-pictures...,—,Criminal,Cat,Domestic Shorthair,NaN,11/15/2022,Adult Cat (5 months-8 years),Healthy In Home,Adopted: Shyanne Baker,6/3/2024,Stray In,9/18/2024,Adoption,108,108,108
3,06/26/2024,04/04/2024,CCL-A-5,Cedar,Domestic Shorthair,—,Female,2Y/5M/8D,Adult Cat,6/4/2023,Brown,NaN,Tabby,992000003226633,NanoCHIP,—,Medical Special Adoption,Adopted: Linay Verge,185,Healthy In Home,No,No,No,FIV+,—,—,https://new-s3.shelterluv.com/profile-pictures...,—,Cedar,Cat,Domestic Shorthair,NaN,6/4/2023,Adult Cat (5 months-8 years),Healthy In Home,Adopted: Linay Verge,4/4/2024,Stray In,10/5/2024,Adoption,185,185,185
4,06/26/2024,06/10/2024,CCL-A-6,Brittle (Max),Domestic Shorthair,—,Female,3Y/7M/10D,Adult Cat,4/2/2022,Calico,NaN,NaN,99200003226732,Other,—,—,Adopted: Jakyrah Harris,101,Healthy In Home,No,No,No,Adult Cat,—,—,https://new-s3.shelterluv.com/profile-pictures...,—,Brittle (Max),Cat,Domestic Shorthair,NaN,4/2/2022,Adult Cat (5 months-8 years),Healthy In Home,Adopted: Jakyrah Harris,6/10/2024,Stray In,9/18/2024,Adoption,101,101,101


In [5]:
# Clean and Prepare Data

## Check column types, is possibly a string at this point
print(df[['Intake Date_x','Outcome Date']].dtypes)
      
## Convert date columns to datetime
df['Intake Date'] = pd.to_datetime(df['Intake Date_x'], errors='coerce')
df['Outcome Date'] = pd.to_datetime(df['Outcome Date'], errors='coerce')

## Check column types, should be a datetime
print(df[['Intake Date','Outcome Date']].dtypes)

## Calculate Length of Stay
df['Length of Stay (days)'] = (df['Outcome Date'] - df['Intake Date']).dt.days

## Remove rows with missing critical data
df = df.dropna(subset=['Intake Date', 'Outcome Date', 'Length of Stay (days)'])

## Remove negative LOS
df = df[df['Length of Stay (days)'] >= 0]

## Take a peak
df[['Animal ID', 'Intake Date', 'Outcome Date', 'Primary Breed_x', 'Sex', 'Age Group', 'Primary Color', 'Adoption Fee Group']].head()


Intake Date_x            object
Outcome Date     datetime64[ns]
dtype: object
Intake Date     datetime64[ns]
Outcome Date    datetime64[ns]
dtype: object


,Animal ID,Intake Date,Outcome Date,Primary Breed_x,Sex,Age Group,Primary Color,Adoption Fee Group
0,CCL-A-1,2024-04-18,2024-07-14,Domestic Shorthair,Female,Senior Cat,Torbie,—
1,CCL-A-2,2024-03-18,2024-10-15,Domestic Shorthair,Male,Adult Cat,Black,Bonded Pair
2,CCL-A-4,2024-06-03,2024-09-18,Domestic Shorthair,Male,Adult Cat,Orange,Adult Cat
3,CCL-A-5,2024-04-04,2024-10-05,Domestic Shorthair,Female,Adult Cat,Brown,FIV+
4,CCL-A-6,2024-06-10,2024-09-18,Domestic Shorthair,Female,Adult Cat,Calico,Adult Cat
